In [1]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost	 = Beta('Cost',0,None,None,0)
Total_TT1	 = Beta('Total_TT1',0,None,None,0)
Total_TT2	 = Beta('Total_TT2',0,None,None,0)
Total_TT3	 = Beta('Total_TT3',0,None,None,0)
Total_TT4	 = Beta('Total_TT4',0,None,None,0)
Dist1	 = Beta('Dist1',0,None,None,0)
Dist2	 = Beta('Dist2',0,None,None,0)
Dist3	 = Beta('Dist3',0,None,None,0)
Dist4	 = Beta('Dist4',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking + Dist1 * distance
Opt2 = Constant2 + Total_TT2 * dur_cycling + Dist2 * distance
Opt3 = Constant3 + Cost * cost_public + Total_TT3 * dur_public + Dist3 * distance
Opt4 = Constant4 + Cost * cost_driving + Total_TT4 * dur_driving + Dist4 * distance
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model2_DistanceSpe"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

           Value  Std err   t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2  -4.63    0.176    -26.4        0         0.219        -21.1   
Constant3  -2.32    0.118    -19.7        0          0.15        -15.4   
Constant4  -1.74    0.109    -15.9        0         0.149        -11.7   
Cost      -0.148   0.0155    -9.59        0        0.0155        -9.58   
Dist1       0.81 0.000196 4.14e+03        0      0.000229     3.54e+03   
Dist2       0.81 0.000123  6.6e+03        0      0.000125     6.49e+03   
Dist3      0.811 7.52e-05 1.08e+04        0      8.49e-05     9.54e+03   
Dist4      0.811 7.57e-05 1.07e+04        0      8.58e-05     9.45e+03   
Total_TT1  -4.78     1.05    -4.55 5.34e-06          1.16        -4.13   
Total_TT2   0.23     1.83    0.126      0.9          1.85        0.124   
Total_TT3  -2.97    0.257    -11.5        0         0.264        -11.2   
Total_TT4  -8.69    0.502    -17.3        0         0.545          -16   

           Rob. p-value  
Constant2  